In [1]:
import numpy as np
import paddle
from paddle_quantum.circuit import UAnsatz

# 设置角度参数 theta = pi
theta = np.array([np.pi])
    
# 我们需要将 Numpy array 转换成 Paddle 中的 Tensor
theta = paddle.to_tensor(theta)

# 设置计算所需的量子比特数量
num_qubits = 1

# 初始化我们的单比特量子电路
cir = UAnsatz(num_qubits)

# 在第一个量子比特(第0号量子比特)的位置上施加一个 Rx 旋转门， 角度为 pi
which_qubit = 0
cir.rx(theta, which_qubit)

# 打印出这个量子门
# 转换成 numpy 
print('量子门的矩阵表达式为：')
print(cir.U.numpy())


ModuleNotFoundError: No module named 'numpy'

In [2]:
import paddle

ModuleNotFoundError: No module named 'paddle'